In [ ]:
#이전 대화는 지워지고 현재 질문만 볼 수 있다.

In [1]:
!pip install pyserial ipywidgets

     |████████████████████████████████| 90 kB 2.4 MB/s eta 0:00:011
You should consider upgrading via the '/home/chat/Python-3.8.12/myenv/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
import serial
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import json

class TemperatureChatbot:
    def __init__(self, port='/dev/ttyACM0', baud_rate=115200):
        self.serial_port = serial.Serial(port, baud_rate, timeout=1)
        self.latest_temp = None
        self.latest_humidity = None
        self.running = True
        
        # Start reading thread
        self.thread = threading.Thread(target=self.read_serial)
        self.thread.daemon = True
        self.thread.start()
        
        # Create UI elements
        self.output = widgets.Output()
        self.text_input = widgets.Text(description='질문:', placeholder='온도가 어때?')
        self.send_button = widgets.Button(description='전송')
        self.send_button.on_click(self.handle_question)
        
        # Display UI
        display(widgets.VBox([self.text_input, self.send_button, self.output]))
        
    def read_serial(self):
        while self.running:
            if self.serial_port.in_waiting:
                line = self.serial_port.readline().decode('utf-8').strip()
                if "Sample OK:" in line:
                    # Parse temperature and humidity
                    parts = line.split("Sample OK: ")[1].split()
                    self.latest_temp = int(parts[0])
                    self.latest_humidity = int(parts[2])
            time.sleep(0.1)
    
    def handle_question(self, button):
        question = self.text_input.value.lower()
        with self.output:
            clear_output()
            print(f"질문: {question}")
            
            if not self.latest_temp or not self.latest_humidity:
                print("센서 데이터를 아직 받지 못했습니다. 잠시만 기다려주세요.")
                return
                
            if '온도' in question:
                print(f"현재 온도는 {self.latest_temp}°C 입니다.")
            elif '습도' in question:
                print(f"현재 습도는 {self.latest_humidity}% 입니다.")
            else:
                print(f"현재 온도는 {self.latest_temp}°C, 습도는 {self.latest_humidity}% 입니다.")
            
        self.text_input.value = ''  # Clear input field
        
    def close(self):
        self.running = False
        self.serial_port.close()
        
# Jupyter Notebook에서 실행할 코드
chatbot = TemperatureChatbot()

In [ ]:
#현재 대화도 저장되고 다음 질문도 저장되는 형태.